In [ ]:
#  
#  Author: Tanish Tyagi
#  

import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

## Loading in List of Regex Expressions

In [ ]:
regex = pd.read_csv(r"keywords.csv") #(r"regex.txt")

In [ ]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == "case"):
        p_list.append(re.compile(k[i]))
    else:
        p_list.append(re.compile(k[i], re.IGNORECASE))
print(len(p_list))

## Loading in Dataset

In [ ]:
john_hsu = pd.read_csv(r"C:\users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Bert\john_hsu_mapping.csv")

In [ ]:
john_hsu = john_hsu[john_hsu["NoteTXT"].notna()]

In [ ]:
john_hsu = john_hsu.reset_index(drop = True)

## Finding Regex Matches

In [ ]:
l = []
for note in tqdm(john_hsu["NoteTXT"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            for i in range(len(m)):
                l.append(m[i])
    
john_hsu["regex_matches"] = l

In [ ]:
# removing all rows that don't have a regex match
john_hsu = john_hsu[john_hsu["regex_matches"] != "[]"]

## Finding Character Locations for all Regex Matches

In [ ]:
locations = []

for r in tqdm(john_hsu['NoteTXT']):
    curr_loc = []
    for p in p_list:
         for match in re.finditer(p ,r):
            curr_loc.append(match.span())
            
    if (curr_loc == []):
        print("STOP")
        break
        
    locations.append(curr_loc)
    
john_hsu["regex_location"] = locations

In [ ]:
john_hsu = john_hsu.reset_index(drop = True)

## Creating Context Windows for all Matches 

In [ ]:
def create_context_windows(i):
    merged = []
    context_length_one_direction = 100
    
    for j in range(len(john_hsu["regex_location"][i])):
        start = john_hsu["regex_location"][i][j][0]
        end = john_hsu["regex_location"][i][j][1]
    
        prune_start = False
        prune_end = False
        if (start - context_length_one_direction <= 0):
            start = 0
            prune_start = True
        if (end + context_length_one_direction >= len(john_hsu["NoteTXT"][i]) - 1):
            end = len(john_hsu["NoteTXT"][i]) - 1
            prune_end = True

        if (prune_start and prune_end):
            merged.append((start, end))
        else:
            if (prune_start):
                merged.append((start, end + context_length_one_direction))
            if (prune_end):
                merged.append((start - context_length_one_direction, end))

        if (not prune_start and not prune_end):
            merged.append((start - context_length_one_direction, end + context_length_one_direction))
    
    return merged

In [ ]:
merged_locs = []
for i in tqdm(range(len(john_hsu))):
     merged_locs.append(create_context_windows(i))

john_hsu["merged_row_location"] = merged_locs

## Merging all Context Windows on the Note Level

In [ ]:
def merge(intervals):  
    ans = []
    i = 1
    intervals.sort()
    tempans = intervals[0]
    
    while i < len(intervals):
        # seeing if context windows overlap with each other - if they do merge them together
        if tempans[0] <= intervals[i][1] and tempans[1] >= intervals[i][0]:
            newtempans = [min(tempans[0] , intervals[i][0]) , max(tempans[1] , intervals[i][1])]
            tempans = newtempans
            i += 1
        else:
            ans.append(tempans)
            tempans = intervals[i]
            i += 1
            
    ans.append(tempans)
    return ans

In [ ]:
merged_locs = []
for i in tqdm(range(len(john_hsu))):
     merged_locs.append(merge(john_hsu["merged_row_location"][i]))

john_hsu["merged_row_location"] = merged_locs        

## Creating Sequences from Merged Context Windows

In [ ]:
def pull_context_windows(i, col):
    sequence = ""
    
    for j in range(len(john_hsu[col][i])):
        if j > 0 and sequence != "":
            sequence += " ----- "
            
        start = john_hsu[col][i][j][0]
        end = john_hsu[col][i][j][1]
        
        sequence += john_hsu["NoteTXT"][i][start : end]
    
    return sequence

In [ ]:
seqs = []

for i in tqdm(range(len(john_hsu))):
    seqs.append(pull_context_windows(i, "merged_row_location"))

john_hsu["regex_sent"] = seqs

## Tokenizing all Sequences into ClincalBERT Tokens

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
def clinical_bert_tokenize(i, col):
    tokens = tokenizer.encode_plus(john_hsu[col][i], add_special_tokens = False, return_tensors = 'pt')

    return (len(tokens['input_ids'][0]))

In [ ]:
token_lens = []

for i in tqdm(range(len(john_hsu))):
    token_lens.append(clinical_bert_tokenize(i, "regex_sent"))

john_hsu["token_length"] = token_lens

## Padding all Sequences such that they are as close as possible to 512 Clincial Bert Tokens

In [ ]:
john_hsu["padded_merged_regex_location"] = john_hsu["merged_row_location"].copy()

for i in tqdm(range(len(john_hsu))):
    john_hsu["padded_merged_regex_location"][i] = list((john_hsu["padded_merged_regex_location"][i]))

In [ ]:
def generate_padded_context_windows(i):
    if (john_hsu["token_length"][i] <= 400):
        locs = list((john_hsu["merged_row_location"][i]))
        length = len(locs)
        start = locs[0][0] 
        end = locs[length - 1][1]
        
        #print(start, end)
                
        seq_length = 1000
        length_to_add_one_side = int((seq_length - john_hsu["sequence_length"][i]) / 2)
        
        # print(length_to_add_one_side, john_hsu["sequence_length"][i])
        
        prune_start = False
        prune_end = False
        if (start - length_to_add_one_side <= 0):
            start = 0
            prune_start = True
        if (end + length_to_add_one_side >= len(john_hsu["NoteTXT"][i]) - 1):
            end = len(john_hsu["NoteTXT"][i]) - 1
            prune_end = True
            
        # print(prune_start, prune_end)

        if (prune_start):
            end += length_to_add_one_side
            if (end + length_to_add_one_side <= len(john_hsu["NoteTXT"][i])):
                end += length_to_add_one_side
        if (prune_end):
            start -= length_to_add_one_side
            if (start - length_to_add_one_side >= 0):
                start -= length_to_add_one_side
        if (not prune_start and not prune_end):
            start -= length_to_add_one_side
            end += length_to_add_one_side
            
        locs = [list(x) for x in locs]
        locs[0][0] = start
        locs[length - 1][1] = end
        
        john_hsu["padded_merged_regex_location"][i] = locs

In [ ]:
for i in tqdm(range(len(john_hsu))):
    generate_padded_context_windows(i)

In [ ]:
padded_seqs = []

for i in tqdm(range(len(john_hsu))):
    padded_seqs.append(pull_context_windows(i, "padded_merged_regex_location"))

john_hsu["padded_regex_sent"] = padded_seqs

## Minor Cleaning to Regex Matches and Sequence columns

In [ ]:
john_hsu['regex_matches'] = john_hsu['regex_matches'].astype(str)
john_hsu['regex_matches'] = john_hsu['regex_matches'].str.replace("\[\]", '')

In [ ]:
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].astype(str)
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace('[', '')
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace(']', '')

In [ ]:
john_hsu = john_hsu.dropna(subset=["padded_regex_sent"])
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace('"', "'")

## Making Sequences more readable by following below regex substitutions:
1. Replace combination of 3 spaces + bullet point + 1 space with semicolon
2. Replace 4 more or spaces with a single space
3. Replace 2 spaces with newline character
4. Replace 4 spaces with 2 newline characters

In [ ]:
def note_preprocessing(note):
    note = re.sub(r'   • ', ' ; ', note)
    note = re.sub(r'[ ]{4,}', ' ', note)
    note = re.sub(r'  ', '\n', note)
    note = re.sub(r'    ', '\n\n', note)
    return note

In [ ]:
for i in tqdm(range(len(john_hsu))):
    john_hsu.at[i, "padded_regex_sent_preprocessed"] = note_preprocessing(john_hsu.at[i, "padded_regex_sent"])

In [ ]:
john_hsu["sequence_length"] = john_hsu['padded_regex_sent_preprocessed'].astype(str).map(len)

## Checking count of ClinicalBERT Tokens for Padded Preprocessed Sequences

In [ ]:
padded_token_lens = []

for i in tqdm(range(len(john_hsu))):
    padded_token_lens.append(clinical_bert_tokenize(i, "padded_regex_sent_preprocessed"))

john_hsu["padded_token_length"] = padded_token_lens

In [ ]:
john_hsu["padded_token_length"].describe()

## Removing Extraneous Sequences

In [ ]:
john_hsu = john_hsu.drop_duplicates(subset = ["padded_regex_sent_preprocessed", "NoteID"], keep = "last").reset_index(drop = True)

## Exporting Dataframe to CSV File

In [ ]:
john_hsu[:1000].to_csv(r"Matches/john_hsu_dataset_1000_padding.csv", index = False)

In [ ]:
john_hsu.to_csv(r"Matches/john_hsu_dataset_matches_padding.csv", index = False)

## Checking Count of Sequences Per Patient

In [ ]:
patient_sequence_count = pd.DataFrame(columns = ["PatientID", "sequence_count"])

In [ ]:
for i in range(len(john_hsu["PatientID"].unique())):
    count = len(john_hsu[john_hsu["PatientID"] == john_hsu["PatientID"].unique()[i]])
    patient_sequence_count.loc[i] = (john_hsu["PatientID"].unique()[i], count)

In [ ]:
patient_sequence_count.to_csv(r"Matches/patient_level_sequence_data.csv", index = False)

In [ ]:
patient_sequence_count = pd.read_csv(r"Matches/patient_level_sequence_data.csv")

In [ ]:
patient_sequence_count["sequence_count"].describe()